# Module 6: Matrixfactorisatie

Je hebt in de vorige module een utility matrix berekend met voorspelde ratings, gegeven een gebruikersmatrix en een filmmatrix met scores voor bepaalde genres.

Wat we nu gaan doen is het omgekeerde. We gaan de gebruikersmatrix en een filmmatrix bepalen, aan de hand van de gegeven ratings (de `target` matrix van het vorige deel). Dit heet _matrixfactorisatie_ (_matrix factorization_). 

Matrixfactorisatie is heel vergelijkbaar met factorisatie van integers. Een vraag die je vast wel eens tegen bent gekomen: Als $a \times b = c$ en we weten dat $c = 9$ wat zijn $a$ en $b$? 

Een antwoord vinden op deze vraag noemen het factoriseren van $c$. Dit is heel vergelijkbaar met het factoriseren van matrices. In ons geval is de vraag: Gegeven $M \cdot U^T = R$ en we weten wat $R$ is (de `target`-matrix), wat zijn de factoren $M$ en $U$?

Een groot verschil (met het factoriseren van integers) is dat er bij het factoriseren van matrices vaak geen exact antwoord is. Dus de vraag wordt: Gegeven $M \cdot U^T = \hat{R}$ wat zijn de waardes voor $M$ en $U$ zodat $\hat{R}$ zo dicht mogelijk $R$ benadert. In dit geval gebruiken we de MSE om te bepalen hoe dicht bij het juiste antwoord we zitten. 

Er zijn meerdere manieren waarop je matrices kan factoriseren. Sommige van deze manieren vergen behoorlijk wat kennis van lineaire algebra. Die aanpakken gaan we nu niet bekijken. Maar, het blijkt dat het algoritmische werkpaard van de machine learning, _gradient descent_, heel geschikt is voor het vinden van de matrixfactoren. Het voordeel van gradient descent is dat het vrij intuïtief is (en te begrijpen is zonder al te diep in de lineaire algebra te duiken). Bovendien is gradient descent een zeer breed toepasbaar algoritme dat sowieso heel nuttig is om te kennen.

Het in één keer wiskundig berekenen van de optimale $M$ en $U$ blijk heel lastig te zijn. Er zitten veel haken en ogen aan, en het is in sommige gevallen stomweg niet mogelijk. Maar het blijkt relatief eenvoudig te zijn om een wiskundige manier te vinden om $M$ en $U$ een heel klein beetje aan te passen zodat de voorspelling $\hat{R}$ een klein beetje beter is dan voor die aanpassing.

Hier maakt gradient descent dankbaar gebruik van. Het algoritme werkt ongeveer zo: Begin met domweg willekeurige waardes invullen voor alle velden in matrices $M$ en $U$. Probeer daarna in kleine stapjes (updates) de $M$ en $U$ zodanig aan te passen dat we steeds dichter bij het antwoord komen.

Deze module bestaat uit drie delen.
- Deel 1: _gradient descent_
- Deel 2: _alternating least squares_
- Deel 3: evaluatie

Dit laatste deel bevat een open opdracht, en om deze goed te doen zal je meer tijd nodig hebben dan voor de andere opdrachten. Houd hier rekening mee in je planning.

Voor je aan begint met gradient descent, laad hieronder eerst de benodigde packages en de data van de vorige module:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from helpers import nice, random_df, display_side_by_side
import answers

data_folder = './data/'
pre_defined_items = pd.read_csv(f'{data_folder}three_genres_movies.csv', index_col = 'movieName')
pre_defined_users = pd.read_csv(f'{data_folder}three_genres_users.csv', index_col = 'userId')
predicted = pre_defined_items @ pre_defined_users.T
ratings = pd.read_csv(f'{data_folder}three_genres_ratings.csv', index_col = 'movieName', dtype={'userId': np.int64})
target = ratings.pivot(columns = 'userId', values = 'value').loc[predicted.index, predicted.columns]

# display data
# display(nice(pre_defined_items))
# display(nice(target.iloc[:15,:15], nan = '&middot;'))

# Deel 1: Gradient descent


## Eerst alleen de gebruikersmatrix leren

Om het probleem iets overzichtelijker te houden, is het makkelijk om nog even gebruik te maken van de gegeven filmmatrix en eerst alleen de gebruikersmatrix $U$ te berekenen (leren). Daarna zullen we pas kijken hoe je beide matrices tegelijk kan leren.

Het is best redelijk om aan te nemen dat we de filmmatrix wel weten, maar de gebruikersmatrix niet: We nemen dan even aan dat we weten tot welke genres de films horen, maar we weten niet welke genrevoorkeuren de gebruikers hebben. Dit laatste moeten we uit de data leren. Dit is een situatie die je in de praktijk prima tegen zou kunnen komen.

We willen dus weten hoe we de gebruikersmatrix moeten aanpassen om ervoor te zorgen dat onze voorspellingen beter worden. Dit houdt in dat we de voorspelde ratings, $\hat{R} = M \cdot U^T$, hebben en we willen weten hoe we $U$ moeten aanpassen om de voorspelde ratings, $\hat{R}$, iets te verbeteren. Het blijkt dat je dit wiskundig kan uitwerken. De afleiding vergt wat calculus en gaat voorbij de stof van deze opdracht. Maar hieronder geven we de vergelijkingen waar je op uitkomt als je de wiskunde helemaal zou uitwerken.

De uitwerking bestaat uit twee stappen:

1. We moeten eerst bepalen hoe ver we er eigenlijk naast zitten. We moeten het verschil weten tussen de voorspelde rating en de target:

    $$
    D = R - \hat{R}
    $$

    <table style= "width:100%"><tr><td style="border:1px solid black">

    **Waarbij je er rekening mee moet houden dat $R$ `nan`-waardes bevat.** Als dit zo is, komt er met deze berekening in Pandas op die plek van de uitkomst $D$ ook een `nan`-waarde te staan. Dit zou problemen geven in de berekening hieronder. Dus alle `nan`'s in $D$ moeten worden vervangen voor een `0`.
    
    </td></tr></table>

2. Vervolgens willen we weten hoe we $U$ kunnen updaten om dat verschil $D$ iets kleiner te maken. De update (een stapje) voor de usermatrix ziet er zo uit:

    $$
    U := U + \alpha \cdot D^T \cdot M
    $$

    Hier is $\alpha$ geen matrix, maar gewoon een getal. Dit is de _learning rate_, een kleine waarde die bepaalt hoe groot de stappen zijn die je neemt bij het updaten van de matrices. Als de $\alpha$ te groot kiest, dan werkt het leren niet goed. Als je hem te klein kiest dan gaat het leren erg traag. Vaak is het een kwestie van uitproberen om hier een goede waarde voor te vinden. Voor deze opdrachten beginnen we met de waarde van $\alpha$ op $0.04$ te zetten.

    De update van de filmmatrix $U$ bevat de het matrix inproduct $D^T \cdot M$. Je hebt dus de gebruikersmatrix $M$ nodig om de nieuwe $U$ te berekenen.

**Als we deze update zouden gebruiken en we berekenen na de update opnieuw de $\hat{R} = M \cdot U^T$ dan is de mean squared error (dus, $\textrm{MSE}(R, \hat{R})$) na de update, als het goed is, net iets lager dan voor de update.**

## Gradient descent in pseudocode

De bovenstaande twee vergelijkingen zijn genoeg voor gradient descent. Gegeven de target $R$ en de gegeven filmmatrix $M$, ziet het algoritme er zo uit:

- Genereer een random gebruikersmatrix, $U$.
- Herhaal $N$ iteraties:
    1. Bereken de utility matrix (voorspel ratings), $\hat{R} = M \cdot U^T$
    2. Bepaal de mse gegeven de huidige voorspelling (de _cost_): $\textrm{cost} = \textrm{mse}(R, \hat{R})$
    3. Update de gebruikersmatrix:
    $$
    D = R - \hat{R} \\
    U := U + \alpha \cdot D^T \cdot M
    $$

Wat goede waardes zijn voor $\alpha$ en $N$ ga je experimenteel vaststellen.

### Vraag 1

\[2 pt.\]

Het grootste gedeelte van het algoritme is hieronder al geïmplementeerd. Het enige wat mist is de wiskunde uit stap 3 voor het updaten van de gebruikersmatrix. 

Implementeer hieronder de functie `update_users`. Deze functie heeft als input de filmmatrix (`movies`), de huidige gebruikersmatrix (`users`) de voorspelling zoals berekend in stap 1 van het algoritme (`prediction`), de target matrix ( `target`) en de learning rate (`alpha`). De functie moet de nieuwe gebruikersmatrix berekenen met de vergelijkingen zoals gegeven in stap 3 van het algoritme.


<table style= "width:100%"><tr><td style="border:1px solid black">

Tip: Vergeet niet dat $R$ een hoop `nan`-waardes bevat. Dit betekent dat $D$ in de vergelijkingen hierboven ook een hoop `nan`-waardes zal bevatten. Deze waardes zijn problematisch voor de rest van het algoritme. Het is dus belangrijk dat je de `nan`-waardes uit $D$ vervangt voor $0$.
    
</td></tr></table>

In [ ]:
def MSE(m1, m2):
    se = (m2 - m1)**2
    return se.mean().mean()

def update_users(movies, users, prediction, target, alpha):
    # TODO
    


def fit_user(target, movies, iterations = 200, alpha = 0.04):
    # compute random user matrix
    users = random_df(target.columns, movies.columns)

    costs = []

    # repeat N times
    for i in tqdm(range(iterations)):
        # 1. predict utility
        prediction = movies @ users.T

        # 2. compute cost (mse)
        cost = MSE(target, prediction)

        # 3. update user matrix
        users = update_users(movies, users, prediction, target, alpha)

        costs.append(cost)
    
    
    return users, costs
    
users, costs = fit_user(target, pre_defined_items, iterations = 100, alpha = 0.04)
    
print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Als je de functie correct hebt geïmplementeerd zou je in de plot moeten zien dat de mse elke iteratie wat kleiner wordt.

Check of je antwoord correct lijkt:

In [ ]:
answers.test_1(costs)

Met de code hieronder kan je delen van de $R$ en $\hat{R}$ naast elkaar bekijken. Komen de voorspelde waardes uit $\hat{R}$ overeen met de gegeven waardes uit $R$? 

Je ziet dat er een hoop `nan`-waardes zijn in $R$, waar $\hat{R}$ wel voorspellingen voor doet. Dat zijn precies de voorspellingen die we uiteindelijk kunnen gebruiken voor het doen van nieuwe aanbevelingen.

In [ ]:
display_side_by_side(nice(target.iloc[:16,:4], nan = '&middot;'), 
                     nice((pre_defined_items @ users.T).iloc[:16,:4]), 
                     titles = ["target", "learned"])

Je kan het algoritme meer iteraties laten draaien om ervoor te zorgen dat de mse nog kleiner wordt. Door de learning rate $\alpha$ groter te maken kan je ervoor zorgen dat het leren sneller gaat, waardoor je minder iteraties nodig hebt om op een lagere mse score uit te komen. Maar, daar is een grens aan, als je de learning rate te hoog instelt werkt het algoritme niet meer.

### Vraag 2

\[1 pt.\]

Experimenteer hieronder met de parameters (`iterations` en `alpha`) in de aanroep van `fit_user`. Zorg ervoor dat de mse uiteindelijk onder de $0.005$ komt te liggen. 

<table style= "width:100%"><tr><td style="border:1px solid black">

Tip: Elke keer dat je het algoritme draait kan de uitkomst een beetje anders zijn. Zodra je goede waardes voor de parameters hebt gevonden, draai het algoritme nog een aantal keer met dezelfde waardes om zeker te zijn dat het altijd een mse onder de $0.005$ geeft.
  
</td></tr></table>

In [ ]:
# TODO


# users, costs = fit_user(target, pre_defined_items, iterations = ???, alpha = ???)

print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Bekijk hieronder weer delen van de $R$ en $\hat{R}$ naast elkaar. Ziet de $\hat{R}$ er zo op het oog beter uit dan dezelfde matrix in vraag 1?

In [ ]:
display_side_by_side(nice(target.iloc[:16,:4], nan = '&middot;'), 
                     nice((pre_defined_items @ users.T).iloc[:16,:4]), 
                     titles = ["target", "learned"])

Test je antwoord:

In [ ]:
answers.test_2(costs)

### Vraag 3

\[1 pt.\]

Wat is de maximale waarde voor de learning rate $\alpha$, waarbij het algoritme nog correct werkt?

YOUR ANSWER HERE

# Deel 2: Alternating least squares

In het vorige algoritme zijn we ervan uitgegaan dat we de genrescores weten voor alle films. Dat soort informatie hebben we alleen lang niet altijd. Bovendien is dat niet altijd een optimale categorisatie van de films: Het is zeer onwaarschijnlijk dat het gebruiken van de genres `drama`, `thriller` en `comedy` de beste manier zou zijn om filmsmaak te karakteriseren. 

Dit kunnen we allemaal oplossen door niet alleen de gebruikersmatrix, maar ook de filmmatrix te leren. Daarvoor kunnen we bijna hetzelfde algoritme gebruiken als hierboven. De voornaamste aanpassing zit hem in de update. Naast een update voor $U$ hebben we ook een update voor $M$ nodig. De update (een stapje) voor de matrices ziet er zo uit:

$$
M := M + \alpha \cdot D \cdot U\\
U := U + \alpha \cdot D^T \cdot M
$$

We hebben hier een update voor $M$ toegevoegd. De update van $U$ is hetzelfde als voorheen. 

Er is hier alleen wel een kleine extra moeilijkheid: De update van de gebruikersmatrix is afhankelijk van de filmmatrix en vice versa. Dat maakt dat we ze niet zomaar tegelijk kunnen updaten.

De truc is om ze niet _tegelijk_ te updaten, maar dit _beurtelings_ te doen: We nemen eerst even aan dat de filmmatrix niet verandert en we updaten alleen de gebruikersmatrix (zoals we ook in de vorige opdracht deden). Vervolgens draaien we de rollen om we nemen aan dat de gebruikersmatrix niet verandert en updaten we de filmmatrix.

Het algoritme ziet er dan zo uit (gegeven de target $R$):

- Genereer een random gebruikersmatrix, $U$ en een random filmmatrix $M$.
- Herhaal N iteraties:
    - fase 1 (update gebruikersmatrix)
        1. Bereken de utility matrix (voorspel ratings), $\hat{R} = M \cdot U^T$
        2. Bepaal de mse gegeven de huidige voorspelling (de _cost_): $\textrm{cost} = \textrm{mse}(R, \hat{R})$
        3. Update de user matrix:
        $$
        D = R - \hat{R} \\
        U := U + \alpha \cdot D^T \cdot M
        $$
    - fase 2 (update filmmatrix)
        4. Bereken de utility matrix (voorspel ratings), $\hat{R} = M \cdot U^T$
        5. Bepaal de mse gegeven de huidige voorspelling (de _cost_): $\textrm{cost} = \textrm{mse}(R, \hat{R})$
        6. Update de film matrix:
        $$
        D = R - \hat{R} \\
        M := M + \alpha \cdot D \cdot U
        $$

Vanwege het afwisselen van de rollen van de gebruikers- en filmmatrix, noemen we dit een *alternating least squares* (ALS) algoritme.

Het is belangrijk om je te beseffen dat we hiermee ook het idee van filmgenres hebben losgelaten. We leren zowel de gebruikers- als de filmmatrix. Omdat we beide random waardes geven in het begin, ligt hier geen specifieke informatie over filmgenres in besloten.

Wat betekenen de scores dan wel? Dat is moeilijk te zeggen. Het is een categorisering die het beste de data voorspelt. Het kan zijn dat een bepaalde categorie vooral hoge scores oplevert voor films met actiefilms, of voor films met Brad Pitt, of films die korter duren dan 100 minuten, of het kan een veel abstractere categorisering zijn die we niet goed semantisch kunnen duiden. Het maakt ook niet uit, zolang het maar goede voorspellingen oplevert. 

Het enige wat we nog kunnen zeggen is dat twee films die vergelijkbare scores voor de verschillende categorieën hebben op elkaar lijken. Ze lijken op elkaar in die zin dat als een gebruiker de ene film goed vindt, de kans groot is de andere ook goed te zullen vinden.

We leren hier dus niet alleen wat de best scores zijn voor de filmcategorieen, maar ook wat beste categorieen zijn om een film mee te beschrijven!

### Vraag 4

\[3 pt.\]

Het alternating least squares algoritme is hieronder al deels geïmplementeerd. Fase 2 van het algoritme is nog niet ingevuld en de functie `update_movies` is nog niet af. Implementeer deze missende onderdelen.

In [ ]:
def update_movies(movies, users, prediction, target, alpha):
    # TODO
    


def fit(target, iterations = 200, alpha = 0.04, number_of_categories = 3):
    # compute random matrices
    users = random_df(target.columns, list(range(number_of_categories))).copy()
    movies = random_df(target.index, list(range(number_of_categories))).copy()

    costs = []

    # repeat N times
    for i in tqdm(range(iterations//2)):
        # phase 1 (update user matrix)
        
        # 1. predict utility
        prediction = movies @ users.T

        # 2. compute cost (mse)
        cost = MSE(target, prediction)
        costs.append(cost)

        # 3. update user matrix
        users = update_users(movies, users, prediction, target, alpha)

        # phase 2 (update item matrix)
 
        # TODO
        
    
    return users, movies, costs
    
users, movies, costs = fit(target, iterations = 100, alpha = 0.04)
    
print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Check je uitkomst:

In [ ]:
answers.test_4(costs)

### Vraag 5

\[1 pt.\]

Experimenteer hieronder met de parameters (`iterations` en `alpha`). Zorg ervoor dat de mse uiteindelijk onder de $0.001$ komt te liggen.

In [ ]:
# TODO



print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Check de uitkomst:

In [ ]:
answers.test_5(costs)

Zoals gezegd komen de kolommen in de gebruikers- en filmmatrices niet meer overeen met specifieke genres, zoals in het eerst deel van deze module. Er is ook geen duidelijke reden meer om specifiek 3 categorieën (kolommen) te hanteren. We kunnen de matrices net zo goed 4 categorieën geven, of 2, of 10. Zolang het aantal kolommen voor beide matrices maar gelijk is. Hoe meer categorieën we toevoegen, hoe meer informatie we kunnen coderen. (Het is overigens niet zo dat meer categorieën ook altijd betere voorspellingen geeft.)

### Vraag 6

\[2 pt.\]

De functie `fit` genereert standaard matrices met 3 categorieën, maar de functie heeft een parameter, `number_of_categories`, waarmee je dit aantal kan aanpassen. Run hieronder de `fit`-functie voor 2, 3 en 4 categorieën met `iterations = 300` en `alpha = 0.1`. 

Plot de `costs` van alle drie de runs in 1 grafiek. Zorg ervoor dat de grafiek duidelijk gelabeld is (gebruik een legenda).

In [ ]:
# TODO


Je weet nu hoe je een matrix kan factoriseren met het alternating least squares algoritme. Dit is een universele techniek die je voor veel meer dan alleen recommender systems kan gebruiken. Elke keer dat je een matrix hebt met missende waardes (nan waardes), kan je in theorie matrixfactorisatie gebruiken om deze waardes te voorspellen. 

We hebben in deze opdracht nog één belangrijk aspect niet behandeld: We hebben de resultaten van deze matrixfactorisatie nog niet geëvalueerd. Je hebt weliswaar de MSE berekend, maar dat hebben we niet gedaan voor een aparte testset. Zoals je in eerder opdrachten hebt gezien moeten we voor de evaluatie echt gebruik maken van zo'n testset. Het gebruik van de MSE in deze opdracht was dus **niet** voor de evaluatie van het algoritme. Deze werd alleen gebruikt voor het updaten van de matrices en het monitoren van de voortgang van dit algoritme.

Maar je kan dezelfde evaluatietechnieken als in de vorige modules natuurlijk ook hier weer gebruiken. Je voorspelt net als in de vorige modules ratings voor films en gebruiker. Je kan hiermee net als in de vorige modules films aanbevelen (door bijvoorbeeld weer een threshold-waarde te kiezen). En deze aanbevelingen kan je weer gebruiken voor het evalueren, met bijvoorbeeld *precision* en *recall*.

### Vraag 7

\[4 pt.\] (uitvoering: 2pt, conclusie 2pt.)

Voor deze opdracht ga je werken met de dataset van module 2. Je kan de volgende commando's gebruiken om de data te laden: 

    training_data = pd.read_pickle(f'./mini-movielens/ratings_t80_training.pkl')
    test_data = pd.read_pickle(f'./mini-movielens/ratings_t80_test.pkl')
    
Wat we willen weten: Wat is _voor deze data_ het beste aantal categoriën? Je kan hiervoor de MSE van de voorspellingen ten opzichte van de _test data_ gebruiken. Dus, concreter, de vraag die je moet beantwoorden: 

**Welk aantal categoriën geeft de laagste MSE voor de test data?** Maak plots om je antwoord te onderbouwen.

Het is aan jou om uit te vogelen hoe je dit gaat doen. Wat ga je plotten? Hoe ga je de data transformeren? Hoe ga je bepalen wat een goede learning rate is? Hoeveel iteraties ga je doen? Schrijf je conclusie in de volgende cel. Schrijf je code in de cel eronder.

In je conclusie vermeld je wat het beste aantal categoriën is en beschrijf je hoe je dat kan afleiden uit de plots die je hebt gemaakt. In de code die je gaat schrijven genereer je de plots die je helpen je argument te onderbouwen. Het kan zijn dat je de `fit` functie een beetje wilt aanpassen. _Doe dat niet in de code hierboven_. Als je de `fit` functie (of een andere functie die je hierboven hebt geschreven) wilt aanpassen, kopieer deze dan in de codecel hieronder en pas het daar aan.

YOUR ANSWER HERE

In [ ]:
# TODO
